In [1]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import json
import numpy as np
import datetime
import hvplot.pandas


import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

In [2]:
load_dotenv()
opensea_api_key = os.getenv('OPENSEA_API_KEY')

# Data for Context and Analysis

### Collection Stats

In [3]:
url = "https://api.opensea.io/api/v2/collections/pudgypenguins/stats"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

collection_stats_response_json = response.json()

collection_totals_dict = collection_stats_response_json["total"]

# I had to set this index because there is not index and it was causing an error
# I used ChatGPT to debug this issue
collection_totals_df = pd.DataFrame(collection_totals_dict, index=[0])

collection_totals_df

,volume,sales,average_price,num_owners,market_cap,floor_price,floor_price_symbol
0,346984.239815,79055,4.38915,4811,98316.267903,12.499,ETH


## Trait Counts

In [4]:
url = "https://api.opensea.io/api/v2/traits/pudgypenguins"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

# Formatting as json
traits_json = response.json()

# get the "counts" and not "categories" key
traits_json = traits_json.get("counts", [])

# preview
traits_json

{'Background': {'Beige': 1152,
  'Blue': 1587,
  'Mint': 1389,
  'Red': 755,
  'Tangerine': 1135,
  'Yellow': 873,
  'Pink': 710,
  'Purple': 1282,
  'On The Beach': 1,
  'Supermarket': 1,
  'Green': 1,
  'Trick Or Treating': 1,
  'Underwater': 1},
 'Skin': {'Light Gray': 1189,
  'Mint': 423,
  'Maroon': 731,
  'Normal': 2649,
  'Leopard Pink': 89,
  'Dark Gray': 1329,
  'Baby Pink': 623,
  'Cream': 646,
  'Olive Green': 705,
  'Red': 344,
  'Gold': 44,
  'Leopard Gray': 89,
  'Ice': 22,
  'Navy Blue': 4,
  'Black': 1},
 'Body': {'Vote 4 Pudgy': 203,
  'Kimono Blue': 108,
  'Scarf Blue': 237,
  'Shirt Blue': 290,
  'Fish Lover': 70,
  'Turtleneck Grey': 210,
  'Lei Pink': 168,
  'Bow Tie Blue': 154,
  'Tank Top Yellow': 193,
  'Puffer Orange': 158,
  'Huddle Shirt': 228,
  'Kimono Brown': 268,
  'Lei Assorted': 76,
  'Bronze Medal': 193,
  'Gold Medal': 58,
  'Heart': 102,
  'Turtleneck Green': 254,
  'Scarf Pink': 177,
  'Surfboard Necklace': 166,
  'Swordman': 62,
  'Christmas Sweate

#### All Traits

In [5]:
# df of all traits
# documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html
# similar to how OneHotEncoder creates new column names
traits_df_normalized = pd.json_normalize(traits_json)

traits_df_normalized

,Background.Beige,Background.Blue,Background.Mint,Background.Red,Background.Tangerine,Background.Yellow,Background.Pink,Background.Purple,Background.On The Beach,Background.Supermarket,...,Head.Sideways Red,Head.Egg Gold,Head.Flower Crown,Head.Fish Orange,Head.Ice Crown,Head.Hatched Gold,Head.Pineapple Suit,Head.Banana Suit,Head.Normal,Head.Shark Suit
0,1152,1587,1389,755,1135,873,710,1282,1,1,...,90,45,66,44,14,9,1,1,1,1


In [6]:
# visualize

# transpose to one column
traits_df_normalized = traits_df_normalized.transpose()

# sort by most common
traits_df_normalized.sort_values([0], ascending=False, inplace=True)

# find top 5
top_10_traits = traits_df_normalized.head(10)



In [7]:
top_10_traits_graph = top_10_traits.hvplot.bar(
    rot=30,
    title="Pudgy Penguin Top 10 Traits",
    ylabel="Count"
)

hvplot.save(top_10_traits_graph, "traits.html")

top_10_traits_graph

:Bars   [index]   (0)

#### Isolating Traits

In [8]:
# show all trait keys
traits_json.keys()

dict_keys(['Background', 'Skin', 'Body', 'Face', 'Head'])

In [9]:
# explode the df
traits_df_exploded =  pd.DataFrame(traits_json)
traits_df_exploded = traits_df_exploded.explode("Background")

# create new dfs for each trait, sorted by values
# if you don't do reset_index(drop=True), it appears as a series
background_trait_df = traits_df_exploded["Background"].dropna().sort_values(ascending=False).reset_index()
skin_trait_df = traits_df_exploded["Skin"].dropna().sort_values(ascending=False).reset_index()
body_trait_df = traits_df_exploded["Body"].dropna().sort_values(ascending=False).reset_index()
face_trait_df = traits_df_exploded["Face"].dropna().sort_values(ascending=False).reset_index()
head_trait_df = traits_df_exploded["Head"].dropna().sort_values(ascending=False).reset_index()

# rename columns and set indices
background_trait_df = background_trait_df.rename(columns={"index":"background", "Background":"count"}).set_index("background")
skin_trait_df = skin_trait_df.rename(columns={"index":"skin", "Skin":"count"}).set_index("skin")
body_trait_df = body_trait_df.rename(columns={"index":"body", "Body":"count"}).set_index("body")
face_trait_df = face_trait_df.rename(columns={"index":"face", "Face":"count"}).set_index("face")
head_trait_df = head_trait_df.rename(columns={"index":"head", "Head":"count"}).set_index("head")

# display trait dataframes
display(background_trait_df, skin_trait_df, body_trait_df, face_trait_df, head_trait_df)

,count
background,
Blue,1587.0
Mint,1389.0
Purple,1282.0
Beige,1152.0
Tangerine,1135.0
Yellow,873.0
Red,755.0
Pink,710.0
On The Beach,1.0


,count
skin,
Normal,2649.0
Dark Gray,1329.0
Light Gray,1189.0
Maroon,731.0
Olive Green,705.0
Cream,646.0
Baby Pink,623.0
Mint,423.0
Red,344.0


,count
body,
Shirt Blue,290.0
Kimono Brown,268.0
Shirt Red,259.0
Turtleneck Green,254.0
Scarf Green,250.0
...,...
Pineapple Suit,1.0
Banana Suit,1.0
Mirrored,1.0


,count
face,
Winking,961.0
Blushing,932.0
Normal,864.0
Circle Glasses,768.0
Cute,588.0
Monocle,496.0
Squad,445.0
Eyepatch,426.0
Cross Eyed,417.0


,count
head,
Headband,357.0
Flat Cap Blue,355.0
Bucket Hat Green,349.0
Backwards Hat Red,294.0
Hat Blue,286.0
Top Hat,282.0
Hat Red,279.0
Beanie Orange,276.0
Flat Cap Tan,274.0


# Data for Model

## 30 Days of Events

In [10]:
# Set today's date
today = datetime.datetime.now()

# set number of days in past
past_date = today - datetime.timedelta(days=30)

# Convert to unix (starting 1970!)
unix_past_date = int(past_date.timestamp())

# Create blank df to store page data
events_df = pd.DataFrame()

# Initialize next_event variable
next_event = None

while True:
    url = f"https://api.opensea.io/api/v2/events/collection/pudgypenguins?after={unix_past_date}&event_type=sale"
    if next_event:
        # This adds the next event value (and required syntax) to the URL string
        url += f"&next={next_event}"

    headers = {
        "accept": "application/json",
        "x-api-key": opensea_api_key
    }

    response = requests.get(url, headers=headers)

    # Formatting as json
    events_json = response.json()

    # Get the "asset events"
    events_data = events_json.get("asset_events", [])
    
    # Store to df
    page_events_df = pd.json_normalize(events_data)
    
    # Concatenate to master df
    events_df = pd.concat([events_df, page_events_df], ignore_index=True)
    
    # Update next_event for pagination
    next_event = events_json.get("next")

    # If there are no more pages, exit the loop
    if not next_event:
        break

# preview
events_df

,event_type,order_hash,chain,protocol_address,closing_date,quantity,seller,buyer,transaction,event_timestamp,...,nft.image_url,nft.metadata_url,nft.opensea_url,nft.updated_at,nft.is_disabled,nft.is_nsfw,payment.quantity,payment.token_address,payment.decimals,payment.symbol
0,sale,0x395f34923316404cb165ae4536d5ef628d0124b3a313...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1715025359,1,0xd200b5124eedcb803c5c04d793f1a06c34b620bb,0xad30abb4c5ad47217137335e90e150738c88e06f,0xee5ca011769daac8a463423b4c2d4a9e896ca01b1f20...,1715025359,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-05-06T19:56:01.706044,False,False,12479980000000000000,0x0000000000000000000000000000000000000000,18,ETH
1,sale,,ethereum,,1715018951,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x8b0d15ac244f6cee45ccf32215f3c9c2e30219c6,0x8e65b6b6821d6779efaabc7c380c62b9a711fdc7126c...,1715018951,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-05-06T18:09:14.487995,False,False,12000000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
2,sale,,ethereum,,1715011439,1,0xd2babccc375e4d10c9e6c4924bfefa83ce55775e,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x8c1c69c7343b56dd03d14a51d4f33e48160aebf8c2cd...,1715011439,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-05-06T16:04:01.766236,False,False,12499000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
3,sale,0x93bb37c7b6a8c1b638054f1b7c8812f516e6c0019b91...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1715010239,1,0xe783b5131a53fa507edab64d232a4e8a9b87bdfb,0xd200b5124eedcb803c5c04d793f1a06c34b620bb,0xcad58be7581754fb891f81e7768ecc929d266a42d138...,1715010239,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-05-06T15:44:01.953340,False,False,12500000000000000000,0x0000000000000000000000000000000000000000,18,ETH
4,sale,,ethereum,,1715005379,1,0xd2babccc375e4d10c9e6c4924bfefa83ce55775e,0x395fe3ff04d374fb07d7612151f37f8cb6273b90,0x790cff8e9941a80b2c233b0d025c4db625d65e8d88ed...,1715005379,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-05-06T14:23:01.733196,False,False,12499000000000000000,0x0000000000000000000000000000000000000000,18,ETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,sale,,ethereum,,1712463515,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x5f1ee29361206f1a129e808736f11598356c6031,0x1e350aa7c265d263277ca47c7290734f5b4a031e21d6...,1712463515,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-13T16:24:01.833414,False,False,11850000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
1350,sale,,ethereum,,1712463503,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0xa69833b9fda816f1bfc79517e7932e64708df0dd,0x83ea58a431f224cc854a1e8198f99bd474c0d1dbecf7...,1712463503,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-29T15:51:26.690060,False,False,11850000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
1351,sale,,ethereum,,1712460971,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x5f1ee29361206f1a129e808736f11598356c6031,0x8053a11ef420b87c662208517d0a24da5fd5a459db09...,1712460971,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-11T01:29:

### Individual NFT Info

#### query parameters from https://docs.opensea.io/reference/get_nft

In [11]:
# PATH PARAMETERS

# note: nft contract is the same for the whole collection

# identify nft contract (public blockchain address)
# this creates an array
protocol_address = events_df["nft.contract"].unique()

# eliminate the null by getting the max of the array
protocol_address = np.max(protocol_address)

# find unique values for chain (currency)
chain = events_df["chain"].unique().tolist()

# write value without list
chain = chain[0]

# token ID in a list for iteration
token_id_list = events_df["nft.identifier"].drop_duplicates().to_list()

In [12]:
# parsing features function
# credit: Sean!

def get_features (nft):
    features = {}
    features["Background"] = nft["traits"][0]["value"]
    features["Skin"] = nft["traits"][1]["value"]
    features["Face"] = nft["traits"][2]["value"]
    features["Head"] = nft["traits"][3]["value"]
    features["Body"] = nft["traits"][4]["value"]
    features["Rarity"] = nft["rarity"]["rank"]
    
    return features

In [13]:
nfts_dict = {}
token_id = token_id_list[0]

for token_id in token_id_list:
    url = f"https://api.opensea.io/api/v2/chain/{chain}/contract/{protocol_address}/nfts/{token_id}"
    headers = {
        "accept": "application/json",
        "x-api-key": opensea_api_key
    }

    response = requests.get(url, headers=headers)

    # Formatting as json
    nfts_json = response.json()
    
    nfts_dict[token_id] = get_features(nfts_json["nft"])

Sean's code if the above does not work because "nft" is missing in one api call.

        nfts_dict = {}
        token_id = token_id_list[0]

        for token_id in token_id_list:
            url = f"https://api.opensea.io/api/v2/chain/{chain}/contract/{protocol_address}/nfts/{token_id}"
            headers = {
                "accept": "application/json",
                "x-api-key": opensea_api_key
            }

            response = requests.get(url, headers=headers)

            # Formatting as json
            nfts_json = response.json()
            if 'nft' in nfts_json:
                nfts_dict[token_id] = get_features(nfts_json["nft"])

In [14]:
nfts_dict

nfts_df = pd.DataFrame(nfts_dict).transpose()

nfts_df

,Background,Skin,Face,Head,Body,Rarity
626,Blue,Cute,Dark Gray,Panda Hat,Crop Top,2143
1578,Mint,Shirt Blue,Baby Pink,Circle Glasses,Macaroni,6552
1682,Normal,Ninja Headband,Purple,Hero Mask Blue,Crop Top,347
8434,Winking,Yellow,Dark Gray,Bronze Medal,Mohawk Green,6898
6855,Beige,Leopard Pink,Flat Cap Blue,Bath Robe,Football,397
...,...,...,...,...,...,...
7370,Beige,Blushing,Sombrero,Red,Tank Top Blue,3880
6992,Maroon,Mint,Handlebar Bear,Top Hat,Poncho,2690
6945,Blue,Maroon,Bucket Hat Green,Eyepatch,Hoodie Pink,4984
7205,Beige,Normal,Shirt Blue,Flat Cap Blue,Monocle,8595


In [15]:
combined_events_df = events_df[["nft.identifier", "closing_date", "payment.quantity"]]

combined_events_df.set_index("nft.identifier", inplace=True)

combined_events_df

,closing_date,payment.quantity
nft.identifier,,
626,1715025359,12479980000000000000
1578,1715018951,12000000000000000000
1682,1715011439,12499000000000000000
8434,1715010239,12500000000000000000
6855,1715005379,12499000000000000000
...,...,...
3235,1712463515,11850000000000000000
8390,1712463503,11850000000000000000
3172,1712460971,11900000000000000000


In [16]:
combined_df = combined_events_df.merge(nfts_df, how="outer", right_index=True, left_index=True)

combined_df.sample(10)

,closing_date,payment.quantity,Background,Skin,Face,Head,Body,Rarity
nft.identifier,,,,,,,,
8875,1714403255,11060000000000000000,Light Gray,Red,Surfboard Necklace,Cross Eyed,Jester's Hat,2379
2463,1714119851,12510000000000000000,Winking,Yellow,Surfboard Necklace,Olive Green,Camo Helmet,4584
6640,1713412655,10110000000000000000,Light Gray,Tangerine,Turtleneck Grey,Cowboy Hat,Cross Eyed,5169
493,1713320663,10050000000000000000,Beige,Normal,Tank Top Yellow,Hat Blue,Squad,7585
6028,1713043031,10620000000000000000,Maroon,Pink,Bucket Hat Green,Handlebar Bear,Hoodie Pink,1635
2127,1713348215,10190000000000000000,Normal,Tangerine,Cream,Hat Blue,Shirt Red,6490
752,1712654327,11620000000000000000,Maroon,Mint,Blushing,Kimono White,Camo Helmet,4132
5796,1713122255,10760000000000000000,Beige,Hat Blue,Surfboard Necklace,Olive Green,Cross Eyed,3407
6644,1714760831,10920000000000000000,Beige,Maroon,Normal,Santa Hat,Hawaiian Shirt,4748


## Encoding Cat Data (Training)

In [17]:
combined_df.dtypes

closing_date         int64
payment.quantity    object
Background          object
Skin                object
Face                object
Head                object
Body                object
Rarity              object
dtype: object

#### Change Wei to Eth so that we can convert data types.

In [18]:
# change payment.quantity (Wei) to float
# we did not use int because of the overfloat error - no difference with float (both numerical)
combined_df[["payment.quantity", "Rarity"]] = combined_df[["payment.quantity", "Rarity"]].astype(float)

# check
combined_df.dtypes

closing_date          int64
payment.quantity    float64
Background           object
Skin                 object
Face                 object
Head                 object
Body                 object
Rarity              float64
dtype: object

In [19]:
# encoding features

# create instance of OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# create a list of columns to encode
cat_columns = ["Background", "Skin", "Face", "Head", "Body"]

# encode columns
encoded = encoder.fit_transform(combined_df[cat_columns])

# create a dataframe with the encoded columns and numerical columns
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_columns))

# concat with numerical columns
encoded_df = pd.concat([combined_df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# drop cat columns
encoded_df.drop(columns=cat_columns, inplace=True)

encoded_df

,closing_date,payment.quantity,Rarity,Background_Baby Pink,Background_Backwards Hat Blue,Background_Beige,Background_Blue,Background_Bow Tie Blue,Background_Bucket Hat Tan,Background_Cute,...,Body_Suit Red,Body_Surfboard Necklace,Body_Swordman,Body_Tank Top Blue,Body_Top Hat,Body_Tribal Necklace,Body_Turtleneck Blue,Body_Turtleneck Pink,Body_Viking Helmet,Body_Villain Mask
0,1713053207,1.120000e+19,8741.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1714367927,1.191000e+19,6938.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1714226351,1.176000e+19,7702.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1714200911,1.188000e+19,7702.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1714144895,1.201000e+19,7702.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,1714672115,1.130000e+19,5014.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1350,1713162587,1.009000e+19,3235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1351,1713142367,1.036000e+19,3235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1352,1713139775,1.075000e+19,3235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## ETH-USD Conversion

## Create static CSV file for model.

In [20]:
encoded_df.to_csv("encoded_data.csv")